In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

jobs = pd.read_csv('C:\Projects\workforce-data-platform\wdp\employee_advisor\data\job_offers.csv')
# jobs.head()

In [ ]:
# TITLE

# get counts of each job title and convert to DataFrame
title_counts = jobs['title'].value_counts().reset_index()

# rename columns and sort by count descending
title_counts.columns = ['title', 'count']
title_counts = title_counts.sort_values(by='count', ascending=False)

# save job title counts to CSV file
title_counts.to_csv('job_title_counts.csv', index=False)

In [16]:
# SKILLS

# generate list of columns to check based on columns with data type int64
cols_to_check = list(jobs.select_dtypes(include=['int64']).columns[jobs.select_dtypes(include=['int64']).apply(lambda x: x.nunique() == 2)])

# get count of 1's in each column and create summary DataFrame
summary_df = pd.DataFrame({'column_name': cols_to_check,
                           'count': jobs[cols_to_check].sum()}).sort_values(by='count', ascending=False)

# save summary DataFrame to CSV file
summary_df.to_csv('skill_counts.csv', index=False)

In [18]:
# SENIORTIY

# create a summary DataFrame
summary_df = pd.DataFrame(jobs['experience_level'].value_counts())

# reset the index and rename the columns
summary_df.reset_index(inplace=True)
summary_df.rename(columns={'index': 'experience_level', 'experience_level': 'count'}, inplace=True)

# sort by count in descending order
summary_df.sort_values(by='count', ascending=False, inplace=True)

# save the summary DataFrame as a CSV file
summary_df.to_csv('experience_level_summary.csv', index=False)

In [21]:
# EMPLOYERS

# get counts of each job title and convert to DataFrame
company_counts = jobs['company_name'].value_counts().reset_index()

# rename columns and sort by count descending
company_counts.columns = ['company', 'count']
company_counts = company_counts.sort_values(by='count', ascending=False)

# save job title counts to CSV file
company_counts.to_csv('company_counts.csv', index=False)

In [4]:
# FIELDS

# Compute the count of each marker_icon value
count_series = jobs['marker_icon'].value_counts()

# Create a new dataframe with marker_icon and count columns
field_counts = pd.DataFrame({'marker_icon': count_series.index, 'count': count_series.values})

# Save the new dataframe as a CSV file
field_counts.to_csv('field_counts.csv', index=False)

In [ ]:
# SKILLS DISTRIBUTION
skill_counts = pd.read_csv('C:\Projects\workforce-data-platform\wdp\employee_advisor\data\skill_counts.csv')

# Sort the data by count in descending order
skill_counts = skill_counts.sort_values(by='count', ascending=False)

# Create a vertical line plot of the counts for all skills
plt.figure(figsize=(15, 5))
plt.plot(skill_counts['count'], marker='o')

# Set the x and y labels, and the title of the plot
plt.xlabel('Skill')
plt.xticks(rotation=90)
plt.ylabel('Count')
plt.title('Distribution of Skills')

# Show the plot
plt.show()

In [ ]:
# FIELD COUNTS

field_counts = pd.read_csv('C:\Projects\workforce-data-platform\wdp\employee_advisor\data\\field_counts2.csv')

# Sort the data by count in descending order and select the top 50 rows
top_fields = field_counts.sort_values(by='count', ascending=False).head(50)

# Reverse the order of the rows so that the largest counts are at the top
top_fields = top_fields.iloc[::-1]

# Create the horizontal bar chart using matplotlib
plt.figure(figsize=(10,15))
plt.barh(y=top_fields['field'], width=top_fields['count'], height=0.4)

# Add the count as a text label at the end of each bar
for i, count in enumerate(top_fields['count']):
    plt.text(count + 10, i, str(count), ha='left', va='center')

# Set the x and y labels, and the title of the plot
plt.xlabel('Count')
plt.ylabel('Field')
plt.title('Distribution of Top Fields')

# Adjust the spacing between the bars
plt.yticks(np.arange(len(top_fields)), top_fields['field'])
plt.subplots_adjust(left=0.0025)

# Show the plot
plt.show()


In [33]:
# SALARY
jobs = pd.read_csv('C:\Projects\workforce-data-platform\wdp\employee_advisor\data\job_offers.csv')

# Count the number of missing values in the 'mid_point_pln' column of the 'jobs' dataframe
# num_missing = jobs['mid_point_pln'].isna().sum()
# print("Number of missing values in 'mid_point_pln':", num_missing)
# 3635

# remove missing values
jobs = jobs.dropna(subset=['currency'])

# remove jobs outside Poland
jobs = jobs[jobs['country_code'] == 'PL']

# create a dictionary to map currencies to conversion factors
currency_factors = {'pln': 1, 'chf': 4.70, 'eur': 4.67, 'gbp': 5.32, 'usd': 4.29}

# Apply currency conversion to salary columns
jobs['salary_from_pln'] = jobs.apply(lambda x: x['salary_from']*currency_factors.get(x['currency'], 1), axis=1)
jobs['salary_to_pln'] = jobs.apply(lambda x: x['salary_to']*currency_factors.get(x['currency'], 1), axis=1)    

# create a new column to hold the mid point salary in PLN
jobs['midpoint_pln'] = jobs.apply(lambda x: (x['salary_from']*currency_factors.get(x['currency'], 1) 
                                               + x['salary_to']*currency_factors.get(x['currency'], 1))/2, axis=1)

# SALARY BY EXPERIENCE (pesymistic, optimistic, midpoint): experience_level
def summarize_dataframe(df, group_col, value_col, output_file):
    # Group the DataFrame by the specified column and calculate summary statistics
    grouped_df = df.groupby(group_col)[value_col].agg(['count', 'mean', 'median', 'min', 'max', 'std'])

    # Rename the columns to include the value column name
    new_cols = [f'{col}_{value_col}' for col in grouped_df.columns]
    grouped_df.columns = new_cols

    # Save the results to a CSV file
    grouped_df.to_csv(output_file)

summarize_dataframe(jobs, 'experience_level', 'salary_from_pln', 'salary_by_experience.csv')


# SALARY BY FIELD (pesymistic, optimistic, midpoint): marker_icon
summarize_dataframe(jobs, 'marker_icon', 'salary_from_pln', 'salary_by_field.csv')


# SALARY BY CITY (pesymistic, optimistic, midpoint): city
summarize_dataframe(jobs, 'city', 'salary_from_pln', 'salary_by_city.csv')


# SALARY BY COMPANY (pesymistic, optimistic, midpoint): company_name
summarize_dataframe(jobs, 'company_name', 'salary_from_pln', 'salary_by_company.csv')


# SALARY BY COMPANY (pesymistic, optimistic, midpoint): company_size
summarize_dataframe(jobs, 'company_size', 'salary_from_pln', 'salary_by_company_size.csv')


# SALARY BY SKILL (pesymistic, optimistic, midpoint): list of first 50 most popular skills
summarize_dataframe(jobs, 'company_size', 'salary_from_pln', 'salary_by_skill_.csv')